In [50]:
import polars as pl # the same as pandas just faster

import configparser
from pathlib import Path
from datetime import datetime
import sys

sys.path.append('../..') # for imports to work
from src.utils import write_files
from src.MC4R import map_mc4r2phenopackets
from src.utils import PolarsUtils

### Get path to notebook from config file

In [51]:
config = configparser.ConfigParser()
config.read('../../data/config/config.cfg')
real_data = True

if real_data:
    path = Path(config.get('Paths', 'mc4r_path')) 
else:
    path = Path(config.get('Paths', 'synth_data_path'))

phenopackets_out = Path(config.get('Paths', 'phenopackets_out'))

### Set Creator Tag

In [52]:
created_by = config.get('Creator', 'creator_tag')

print(f'Creator tag: {created_by}')

Creator tag: P. Robinson, MD, D. Danis, PhD, A. Graefe, F. Rehburg


### Read data in

In [53]:
df = pl.read_csv(path)
df.head(5)

record_id,sct_422549004,sct_399423000,sct_184099003_y,sct_281053000,sct_263495000,sct_315354004,sct_364699009,sct_278844005,sct_399753006,sct_420259009,sct_767023003,sct_184305005_rd,sct_16100001,sct_16100001_source,sct_769681006_center,sct_440377005,sct_276239002_aim,sct_424850005,sct_405795006_y,sct_405795006_m,sct_405795006_d,sct_423493009,sct_432213005_y,sct_432213005_m,sct_432213005_d,sct_717800004,sct_412726003,sct_64245008,sct_125679009_proposit,sct_842009,sct_439401001_orpha,sct_439401001_orpha_sub,sct_439401001_icd10gm,sct_439401001_alphaid,sct_439401001_alphaidstr,sct_38866009,…,sct_82101005_10_age,sct_82101005_10_gender,sct_75226009_rd,sct_75226009_1_age,sct_75226009_1_gender,sct_75226009_2_age,sct_75226009_2_gender,sct_75226009_3_age,sct_75226009_3_gender,sct_75226009_4_age,sct_75226009_4_gender,sct_75226009_5_age,sct_75226009_5_gender,sct_75226009_6_age,sct_75226009_6_gender,sct_75226009_7_age,sct_75226009_7_gender,sct_75226009_8_age,sct_75226009_8_gender,sct_75226009_9_age,sct_75226009_9_gender,sct_75226009_10_age,sct_75226009_10_gender,sct_309370004_research,sct_309370004_narse,sct_309370004_data,sct_309370004_eu,sct_309370004_int,sct_309370004_aff,sct_309370004_gen,sct_309370004_case,sct_123038009,sct_840566006,sct_840566006_specific,sct_21134002_class,sct_21134002_score,erker_v15_complete
i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""FA062""",null,2001,"""sct_248153007""",null,null,"""sct_90027003""","""CS_MII_Person_…",null,null,"""sct_41847000""","""sct_1220561009…",null,null,"""2007-13-24""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_410672004""",2008,1,23,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_373066001""","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
2,"""YA074""",null,2002,"""sct_248152002""",null,null,"""sct_372148003_…","""CS_MII_Person_…",null,null,"""sct_41847000""","""sct_1220561009…",null,null,"""2011-10-21""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_410672004""",2011,4,18,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
3,"""SA099""",null,2002,"""sct_248153007""",null,null,"""sct_14045001""","""CS_MII_Person_…",null,null,"""sct_41847000""","""sct_1220561009…",null,null,"""2002-7-23""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_261665006""",null,null,null,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
4,"""EA102""",null,2007,"""sct_248152002""

#### Get some info about the data

#### Get number of rows and columns

In [54]:
print(f'Number of rows: {PolarsUtils.get_num_rows(df)}')
print(f'Number of cols: {PolarsUtils.get_num_cols(df)}')

Number of rows: 317
Number of cols: 98


In [55]:
df.describe()

describe,record_id,sct_422549004,sct_399423000,sct_184099003_y,sct_281053000,sct_263495000,sct_315354004,sct_364699009,sct_278844005,sct_399753006,sct_420259009,sct_767023003,sct_184305005_rd,sct_16100001,sct_16100001_source,sct_769681006_center,sct_440377005,sct_276239002_aim,sct_424850005,sct_405795006_y,sct_405795006_m,sct_405795006_d,sct_423493009,sct_432213005_y,sct_432213005_m,sct_432213005_d,sct_717800004,sct_412726003,sct_64245008,sct_125679009_proposit,sct_842009,sct_439401001_orpha,sct_439401001_orpha_sub,sct_439401001_icd10gm,sct_439401001_alphaid,sct_439401001_alphaidstr,…,sct_82101005_10_age,sct_82101005_10_gender,sct_75226009_rd,sct_75226009_1_age,sct_75226009_1_gender,sct_75226009_2_age,sct_75226009_2_gender,sct_75226009_3_age,sct_75226009_3_gender,sct_75226009_4_age,sct_75226009_4_gender,sct_75226009_5_age,sct_75226009_5_gender,sct_75226009_6_age,sct_75226009_6_gender,sct_75226009_7_age,sct_75226009_7_gender,sct_75226009_8_age,sct_75226009_8_gender,sct_75226009_9_age,sct_75226009_9_gender,sct_75226009_10_age,sct_75226009_10_gender,sct_309370004_research,sct_309370004_narse,sct_309370004_data,sct_309370004_eu,sct_309370004_int,sct_309370004_aff,sct_309370004_gen,sct_309370004_case,sct_123038009,sct_840566006,sct_840566006_specific,sct_21134002_class,sct_21134002_score,erker_v15_complete
str,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""",98.0,"""98""","""98""",98.0,"""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""",98.0,98.0,98.0,"""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""",…,"""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98"""
"""null_count""",0.0,"""0""","""98""",0.0,"""0""","""98""","""98""","""3""","""0""","""98""","""98""","""0""","""0""","""98""","""98""","""0""","""0""","""98""","""0""","""98""","""98""","""98""","""0""",5.0,5.0,5.0,"""0""","""98""","""0""","""0""","""0""","""0""","""98""","""0""","""0""","""98""",…,"""98""","""98""","""0""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""98""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""98""","""0""","""98""","""98"""
"""mean""",49.5,null,null,2000.102041,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2010.096774,5.645161,15.075269,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",28.434134,null,null,7.157276,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5.039118,3.181622,8.106944,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""",1.0,"""AA009""",null,1988.0,"""sct_248152002""",null,null,"""sct_14045001""","""CS_MII_Person_…",null,null,"""sct_255398004""","""sct_1220561009…",null,null,"""1988-10-21""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_261665006""",2000.0,1.0,1.0,"""sct_1220561009…",null,"""

#### Display unique values for each column

In [56]:
df.unique()

record_id,sct_422549004,sct_399423000,sct_184099003_y,sct_281053000,sct_263495000,sct_315354004,sct_364699009,sct_278844005,sct_399753006,sct_420259009,sct_767023003,sct_184305005_rd,sct_16100001,sct_16100001_source,sct_769681006_center,sct_440377005,sct_276239002_aim,sct_424850005,sct_405795006_y,sct_405795006_m,sct_405795006_d,sct_423493009,sct_432213005_y,sct_432213005_m,sct_432213005_d,sct_717800004,sct_412726003,sct_64245008,sct_125679009_proposit,sct_842009,sct_439401001_orpha,sct_439401001_orpha_sub,sct_439401001_icd10gm,sct_439401001_alphaid,sct_439401001_alphaidstr,sct_38866009,…,sct_82101005_10_age,sct_82101005_10_gender,sct_75226009_rd,sct_75226009_1_age,sct_75226009_1_gender,sct_75226009_2_age,sct_75226009_2_gender,sct_75226009_3_age,sct_75226009_3_gender,sct_75226009_4_age,sct_75226009_4_gender,sct_75226009_5_age,sct_75226009_5_gender,sct_75226009_6_age,sct_75226009_6_gender,sct_75226009_7_age,sct_75226009_7_gender,sct_75226009_8_age,sct_75226009_8_gender,sct_75226009_9_age,sct_75226009_9_gender,sct_75226009_10_age,sct_75226009_10_gender,sct_309370004_research,sct_309370004_narse,sct_309370004_data,sct_309370004_eu,sct_309370004_int,sct_309370004_aff,sct_309370004_gen,sct_309370004_case,sct_123038009,sct_840566006,sct_840566006_specific,sct_21134002_class,sct_21134002_score,erker_v15_complete
i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
3,"""SA099""",null,2002,"""sct_248153007""",null,null,"""sct_14045001""","""CS_MII_Person_…",null,null,"""sct_41847000""","""sct_1220561009…",null,null,"""2002-7-23""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_261665006""",null,null,null,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
5,"""AA070""",null,2000,"""sct_248152002""",null,null,"""sct_372148003_…","""CS_MII_Person_…",null,null,"""sct_41847000""","""sct_1220561009…",null,null,"""2009-4-17""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_410672004""",2009,7,5,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
22,"""YD053""",null,1993,"""sct_248153007""",null,null,"""sct_372148003_…","""CS_MII_Person_…",null,null,"""sct_41847000""","""sct_1220561009…",null,null,"""2000-17-8""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_410672004""",2001,5,15,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
33,"""EH076""",null,2005,"""sct_248152002""

#### Get number of null values for each column

In [57]:
PolarsUtils.null_value_analysis(df, verbose=True)

There are 230/317 columns with only null values in the data
There are 263/317 columns with at least one null value in the data


variable,null_count,all_null
str,u32,bool
"""sct_399423000""",98,false
"""sct_263495000""",98,false
"""sct_315354004""",98,false
"""sct_364699009""",3,false
"""sct_399753006""",98,false
"""sct_420259009""",98,false
"""sct_16100001""",98,false
"""sct_16100001_s…",98,false
"""sct_276239002_…",98,false


## Preprocessing
- remove null colus
- clean up data
- add id col to data (col name `'mc4r_id'`)

### Remove null cols

In [58]:
df = PolarsUtils.drop_null_cols(df, remove_all_null=True, remove_any_null=False)
df.head(5)

Dropped 230 columns. 87 columns remaining.


record_id,sct_422549004,sct_184099003_y,sct_281053000,sct_364699009,sct_278844005,sct_767023003,sct_184305005_rd,sct_769681006_center,sct_440377005,sct_424850005,sct_423493009,sct_432213005_y,sct_432213005_m,sct_432213005_d,sct_717800004,sct_64245008,sct_125679009_proposit,sct_842009,sct_439401001_orpha,sct_439401001_icd10gm,sct_439401001_alphaid,sct_432213005,sct_263493007,sct_116694002,sct_246454002,sct_439401001_ver,sct_103330002,sct_439401001_val,sct_39154008_hpo,sct_406522009,ln_48007_9,ln_48007_9_mitoch,sct_439401001_gen_val,ln_ll4048_6,sct_439401001_omim_g_1,sct_439401001_omim_g_2,…,ln_48018_6_2,ln_48002_0_2,ln_48019_4_2,ln_53037_8_2,ln_62374_4_3,sct_55446002_str_3,ln_48004_6_3,ln_48005_3_3,ln_48018_6_3,ln_48002_0_3,ln_48019_4_3,ln_53037_8_3,sct_8116006_1,sct_8116006_1_date,sct_8116006_2,sct_8116006_2_date,sct_8116006_3,sct_8116006_3_date,sct_8116006_4,sct_8116006_4_date,sct_72705000_rd,sct_160430005,sct_66839005_rd,sct_160436004,sct_82101005_rd,sct_75226009_rd,sct_309370004_research,sct_309370004_narse,sct_309370004_data,sct_309370004_eu,sct_309370004_int,sct_309370004_aff,sct_309370004_gen,sct_309370004_case,sct_123038009,sct_840566006,sct_21134002_class
i64,str,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""FA062""",2001,"""sct_248153007""","""sct_90027003""","""CS_MII_Person_…","""sct_41847000""","""sct_1220561009…","""2007-13-24""","""sct_1220561009…","""sct_261665006""","""sct_410672004""",2008,1,23,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_373066001""","""ORPHA:71529""","""E66.8""","""sct_1220561009…","""2008-01-23""","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""hl7_conditionv…","""sct_373067005""","""sct_439401001_…","""sct_373066001""","""sct_1220561009…","""ln_LA6706-1""","""sct_1220561009…","""sct_373066001""","""sct_1220561009…",155541.0024,null,…,"""HGNC:6932""",null,null,null,"""ln_LA26806-2""",null,null,null,"""HGNC:6932""",null,null,null,"""HP:0025500""","""2003-06-24""","""HP:0025501""","""2001-08-24""","""HP:0025499""","""2006-10-24""",null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…"
2,"""YA074""",2002,"""sct_248152002""","""sct_372148003_…","""CS_MII_Person_…","""sct_41847000""","""sct_1220561009…","""2011-10-21""","""sct_1220561009…","""sct_261665006""","""sct_410672004""",2011,4,18,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""ORPHA:71529""","""E66.8""","""sct_1220561009…","""2011-04-18""","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""hl7_conditionv…","""sct_373067005""","""sct_439401001_…","""sct_373066001""","""sct_1220561009…","""ln_LA6706-1""","""sct_1220561009…","""sct_373066001""","""sct_1220561009…",155541.0024,null,…,"""HGNC:6932""",null,null,null,"""ln_LA26806-2""",null,null,null,"""HGNC:6932""",null,null,null,"""HP:0025499""","""2004-07-21""",null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…"
3,"""SA099""",2002,"""sct_248153007""","""sct_14045001""","""CS_MII_Person_…","""sct_41847000""","""sct_1220561009…","""2002-7-23""","""sct_1220561009…","""sct_261665006""","""sct_261665006""",null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_

### Update id column
1. Drop old ID (has duplicate values)
2. Initialize new ID column

In [59]:
df.drop_in_place('record_id');

In [60]:
df = PolarsUtils.add_id_col(df, id_col_name='mc4r_id')
df.head(5)

mc4r_id,sct_422549004,sct_184099003_y,sct_281053000,sct_364699009,sct_278844005,sct_767023003,sct_184305005_rd,sct_769681006_center,sct_440377005,sct_424850005,sct_423493009,sct_432213005_y,sct_432213005_m,sct_432213005_d,sct_717800004,sct_64245008,sct_125679009_proposit,sct_842009,sct_439401001_orpha,sct_439401001_icd10gm,sct_439401001_alphaid,sct_432213005,sct_263493007,sct_116694002,sct_246454002,sct_439401001_ver,sct_103330002,sct_439401001_val,sct_39154008_hpo,sct_406522009,ln_48007_9,ln_48007_9_mitoch,sct_439401001_gen_val,ln_ll4048_6,sct_439401001_omim_g_1,sct_439401001_omim_g_2,…,ln_48018_6_2,ln_48002_0_2,ln_48019_4_2,ln_53037_8_2,ln_62374_4_3,sct_55446002_str_3,ln_48004_6_3,ln_48005_3_3,ln_48018_6_3,ln_48002_0_3,ln_48019_4_3,ln_53037_8_3,sct_8116006_1,sct_8116006_1_date,sct_8116006_2,sct_8116006_2_date,sct_8116006_3,sct_8116006_3_date,sct_8116006_4,sct_8116006_4_date,sct_72705000_rd,sct_160430005,sct_66839005_rd,sct_160436004,sct_82101005_rd,sct_75226009_rd,sct_309370004_research,sct_309370004_narse,sct_309370004_data,sct_309370004_eu,sct_309370004_int,sct_309370004_aff,sct_309370004_gen,sct_309370004_case,sct_123038009,sct_840566006,sct_21134002_class
i64,str,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
0,"""FA062""",2001,"""sct_248153007""","""sct_90027003""","""CS_MII_Person_…","""sct_41847000""","""sct_1220561009…","""2007-13-24""","""sct_1220561009…","""sct_261665006""","""sct_410672004""",2008,1,23,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_373066001""","""ORPHA:71529""","""E66.8""","""sct_1220561009…","""2008-01-23""","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""hl7_conditionv…","""sct_373067005""","""sct_439401001_…","""sct_373066001""","""sct_1220561009…","""ln_LA6706-1""","""sct_1220561009…","""sct_373066001""","""sct_1220561009…",155541.0024,null,…,"""HGNC:6932""",null,null,null,"""ln_LA26806-2""",null,null,null,"""HGNC:6932""",null,null,null,"""HP:0025500""","""2003-06-24""","""HP:0025501""","""2001-08-24""","""HP:0025499""","""2006-10-24""",null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…"
1,"""YA074""",2002,"""sct_248152002""","""sct_372148003_…","""CS_MII_Person_…","""sct_41847000""","""sct_1220561009…","""2011-10-21""","""sct_1220561009…","""sct_261665006""","""sct_410672004""",2011,4,18,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""ORPHA:71529""","""E66.8""","""sct_1220561009…","""2011-04-18""","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""hl7_conditionv…","""sct_373067005""","""sct_439401001_…","""sct_373066001""","""sct_1220561009…","""ln_LA6706-1""","""sct_1220561009…","""sct_373066001""","""sct_1220561009…",155541.0024,null,…,"""HGNC:6932""",null,null,null,"""ln_LA26806-2""",null,null,null,"""HGNC:6932""",null,null,null,"""HP:0025499""","""2004-07-21""",null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…"
2,"""SA099""",2002,"""sct_248153007""","""sct_14045001""","""CS_MII_Person_…","""sct_41847000""","""sct_1220561009…","""2002-7-23""","""sct_1220561009…","""sct_261665006""","""sct_261665006""",null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_12

### Parsing step

In [61]:
from src.MC4R import zygosity_map_erker2phenopackets, sex_map_erker2phenopackets
from src.MC4R.ParseMC4R import parse_date_of_diagnosis, parse_year_of_birth, parse_phenotyping_date

# sct_184099003_y (year of birth)
df = PolarsUtils.map_col(df, map_from='sct_184099003_y', map_to='parsed_year_of_birth',\
    mapping=parse_year_of_birth)

# sct_281053000 (sex)
df = PolarsUtils.map_col(df, map_from='sct_281053000', map_to='parsed_sex',\
    mapping=sex_map_erker2phenopackets)

# sct_432213005 (date of diagnosis)
df = PolarsUtils.map_col(df, map_from='sct_432213005',\
    map_to='parsed_date_of_diagnosis' ,mapping=parse_date_of_diagnosis)

# ln_48007_9 (zygosity)
df = PolarsUtils.map_col(df, map_from='ln_48007_9', map_to='parsed_zygosity',\
    mapping=zygosity_map_erker2phenopackets)

# sct_439401001_orpha (diagnosis (ORPHA))
# does not require mapping

# ln_81290_9_1, ln_81290_9_2, ln_81290_9_3 (mutation p.HGVS)
# does not require mapping

# ln_48004_6_1, ln_48004_6_2, ln_48004_6_3 (mutation c.HGVS)
# does not require mapping

# ln_48018_6_1, ln_48018_6_2, ln_48018_6_3 (gene HGNC)
# does not require mapping

# sct_8116006_1, sct_8116006_2, sct_8116006_3, sct_8116006_4, sct_8116006_5 (phenotype\
  #classification
# does not require mapping

# sct_8116006_1_date, sct_8116006_2_date, sct_8116006_3_date, sct_8116006_4_date, \
    # sct_8116006_5_date (dates of phenotype determination)
df = PolarsUtils.map_col(df, map_from='sct_8116006_1_date', map_to='parsed_dates_of_phenotyping1', mapping=parse_phenotyping_date)
df = PolarsUtils.map_col(df, map_from='sct_8116006_2_date', map_to='parsed_dates_of_phenotyping2', mapping=parse_phenotyping_date)
df = PolarsUtils.map_col(df, map_from='sct_8116006_3_date', map_to='parsed_dates_of_phenotyping3', mapping=parse_phenotyping_date)
df = PolarsUtils.map_col(df, map_from='sct_8116006_4_date', map_to='parsed_dates_of_phenotyping4', mapping=parse_phenotyping_date)
df = PolarsUtils.map_col(df, map_from='sct_8116006_5_date', map_to='parsed_dates_of_phenotyping5', mapping=parse_phenotyping_date)


ComputeError: TypeError: parse_date_of_diagnosis() missing 2 required positional arguments: 'month' and 'day'

## Map to phenopackets

In [ ]:
phenopackets = map_mc4r2phenopackets(df, created_by=created_by)

TypeError: Timestamp() takes no arguments

## Write to json

In [ ]:
cur_time = datetime.now().strftime("%Y%m%d-%H%M%S")
phenopackets_out_dir = phenopackets_out / cur_time

write_files(phenopackets, phenopackets_out_dir)